# save the images as video
# Basler ace camera

In [6]:
# import packages and functions
from pypylon import pylon
from imageio import get_writer
import numpy as np
import os
import cv2
from datetime import datetime
import matplotlib.pyplot as plt
import sys
from camcommands import camOpen
from multiprocessing import Process

connect camera with OpenCV

In [3]:
# list cameras connected with PC
# 0. set number of cameras to use
cameras_2_use = 2

# 1. get transport layer and all attached devices
tl_factory = pylon.TlFactory.GetInstance()
devices = tl_factory.EnumerateDevices() #enumerate devices connected

# 1.1 soft ware issue from pylon. this block may be deleted later if the issue is solved
# here, get instance and enumerate devices until two different cameras are recognized
temp_dev_list = []
for i, dev_info in enumerate(devices):
    temp_dev_list.append(dev_info.GetSerialNumber())
while temp_dev_list[0] == temp_dev_list[1]:
    tl_factory = pylon.TlFactory.GetInstance()
    devices = tl_factory.EnumerateDevices()
    temp_dev_list = []
    for i, dev_info in enumerate(devices):
        temp_dev_list.append(dev_info.GetSerialNumber())
    print(temp_dev_list)
    if temp_dev_list[0] != temp_dev_list[1]:
        break

print(temp_dev_list)
"""
# 1.1 software issue from pylon. this block may be deleted later
# remove duplicated camera list
set_devices = []
for device in devices:
    if device not in set_devices:
        set_devices.append(device)
    else:
        pass
set_devices = tuple(set_devices)
"""

# 2. check the devices information
if len(devices) == 0:
    raise pylon.RuntimeException("No camera connected")
else:
    print("total number of devices:", len(devices)) # print total number of devices connected
    for i, dev_info in enumerate(devices): # 
        if dev_info.GetDeviceClass() == 'BaslerGigE': # check if the connected cameras are GigE model. Otherwise software trigger won't work
            print("using %s as IP %s SN %s" % (dev_info.GetModelName(), dev_info.GetIpAddress() ,dev_info.GetSerialNumber()))
        else:
            raise EnvironmentError("no GigE device found")

# 3. create instant array and attach all devices on the Array
cams = pylon.InstantCameraArray(min(len(devices), cameras_2_use)) # create instant
for i, cam in enumerate(cams):
    cam.Attach(tl_factory.CreateDevice(devices[i]))

['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21585923']
['21585923', '21566554']
['21585923', '21566554']
total number of devices: 2
using acA1300-30gc as IP 169.254.4.51 SN 21585923
using acA1300-30gc as IP 169.254.91.231 SN 21566554


In [4]:
# setting camera's parameters
fps = 10 # frame per sec
ExposureTime = 2000 # in µs
Height, width = 962, 1286 # in pixel

for cam in cams:
    cam.Open() # open camera to change the parameter
    print("Setting device ", cam.GetDeviceInfo().GetFriendlyName())
    cam.Height.SetValue(Height)
    cam.Width.SetValue(width)
    cam.ExposureTimeRaw.SetValue(ExposureTime)
    cam.AcquisitionFrameRateAbs.SetValue(fps)
    print(cam.Height.GetValue(), cam.Width.GetValue(), cam.ExposureTimeRaw.GetValue(), cam.AcquisitionFrameRateAbs.GetValue())

    cam.Close() # close camera
    

Setting device  acA1300_SN03 (21585923)
962 1286 2000 10.0
Setting device  acA1300_SN01 (21566554)
962 1286 2000 10.0


In [18]:
# capturing video with multiple cameras at simultaneously
video_pos = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    with get_writer(os.path.join(file_dir, filename)) as writer:
        print("recording start with %s at %s" % (cam.DeviceInfo.GetFriendlyName(), datetime.now()))
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(100)
        while caminstance.IsGrabbing():
            res = caminstance.RetrieveResult(100)
            writer.append_data(res.Array)
            res.Release()
        print("recording finish with %s at %s" % (cam.DeviceInfo.GetFriendlyName(), datetime.now()))

cam1 = Process(target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[0]), cams[0]))
cam2 = Process(target=video_recording, 
                args=("/Users/dkim/Desktop/basler_camera/recording", "test_multicore_%s.avi" % (video_pos[1]), cams[1]))
cam1.start()
cam2.start()
cam1.join()
cam2.join()

recording start with acA1300_SN01 (21566554) at 2021-11-02 16:06:10.753732
recording start with acA1300_SN01 (21566554) at 2021-11-02 16:06:10.760551


In [75]:
# capturing video with multiple cameras at simultaneously
filename = {0:"front", 1:"side"}

#function for simple video recording
def video_recording(file_dir, filename, caminstance):
    with get_writer(os.path.join(file_dir, filename)) as writer:
        
        caminstance.StopGrabbing()
        caminstance.StartGrabbingMax(100)
        while caminstance.IsGrabbing():
            res = caminstance.RetrieveResult(100)
            writer.append_data(res.Array)
            res.Release()

for i, cam in enumerate(cams):
    with get_writer("/Users/dkim/Desktop/basler_camera/recording/%s_multi_cams.avi" % (filename[i] ,) ) as writer:
        cam.StopGrabbing()
        cam.StartGrabbingMax(100)
        while cam.IsGrabbing():
            res = cam.RetrieveResult(100)
            writer.append_data(res.Array)
            #print(res.BlockID)
            res.Release()

RuntimeException: Could not apply configuration. Pylon::GenericException caught in OnOpened method msg=Device doesn't support action commands. The ActionCommand features are not writable. : RuntimeException thrown (file 'ActionTriggerConfiguration.h', line 124) : RuntimeException thrown (file 'ActionTriggerConfiguration.h', line 212)

In [17]:
"""
working with multiple cameras
for the detail commands, see here: https://docs.baslerweb.com/action-commands.html#action-device-key
short paramer description
Action Device Key: 32-bit key to excute a corresponding action on camera
Action Group Key: 32-bit key for a group of devices (cameras) to excute an action
Action Group Mask: 32-bit key for a subgroup of devies

setting up the connection with multiple cameras and checking the parameters
"""

# 1. set action key, group key and group mask. As I'm using one group and all cameras in this group working simaltaneously,
# I did not set multiple action key, group key and group mask
action_key = 0x4711
group_key = 0x112233
group_mask = pylon.AllGroupMask

# 2. Initiate automatic configuration by registering ActionTriggerConfiguration.
for cam in cams:
    cam.RegisterConfiguration(
        pylon.ActionTriggerConfiguration(action_key, group_key, group_mask),
        pylon.RegistrationMode_Append,
        pylon.Cleanup_Delete
        )

# 3. Create a suitable ActionCommand object. For that a GigETransportLayer object is needed.
gige_tl = tl_factory.CreateTl('BaslerGigE')

# 4. Using default value of "255.255.255.255" for fourth parameter 'broadcastAddress'.
act_cmd = gige_tl.ActionCommand(action_key, group_key, group_mask)